In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from datasets import load_dataset
import torch
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

2024-09-25 08:21:34.308851: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 08:21:34.308905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 08:21:34.309804: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 08:21:34.316248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 08:21:35.143452: W tensorflow/compiler/tf2

Load dataset

In [3]:
dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
print(dataset['text'][0])

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

Quantization of models in 4bit NF for Qlora

In [5]:
model_name = "bigscience/bloom-3b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [6]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [7]:
print(model.state_dict().keys())

odict_keys(['transformer.word_embeddings.weight', 'transformer.word_embeddings_layernorm.weight', 'transformer.word_embeddings_layernorm.bias', 'transformer.h.0.input_layernorm.weight', 'transformer.h.0.input_layernorm.bias', 'transformer.h.0.self_attention.query_key_value.weight', 'transformer.h.0.self_attention.query_key_value.bias', 'transformer.h.0.self_attention.query_key_value.weight.absmax', 'transformer.h.0.self_attention.query_key_value.weight.quant_map', 'transformer.h.0.self_attention.query_key_value.weight.quant_state.bitsandbytes__nf4', 'transformer.h.0.self_attention.dense.weight', 'transformer.h.0.self_attention.dense.bias', 'transformer.h.0.self_attention.dense.weight.absmax', 'transformer.h.0.self_attention.dense.weight.quant_map', 'transformer.h.0.self_attention.dense.weight.quant_state.bitsandbytes__nf4', 'transformer.h.0.post_attention_layernorm.weight', 'transformer.h.0.post_attention_layernorm.bias', 'transformer.h.0.mlp.dense_h_to_4h.weight', 'transformer.h.0.mlp

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [9]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 8

In [10]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

Training arguments for the model

In [18]:
output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

### SFTTrainer Class

`SFTTrainer` is a wrapper around the `transformers.Trainer` class, designed specifically for supervised fine-tuning (SFT). It handles proper initialization of the `PeftModel` when a `PeftConfig` object is passed.

#### Args:
- **model** (`Union[transformers.PreTrainedModel, nn.Module, str]`):  
  The model to train, which can be a `PreTrainedModel`, a `torch.nn.Module`, or a string specifying the model name to load from cache or download. If a `PeftConfig` is provided, the model is converted to a `PeftModel`.

- **args** (`Optional[SFTConfig]`):  
  Configuration arguments for the trainer. Defaults to a basic instance of `SFTConfig` with `output_dir` set to a temporary directory (`tmp_trainer`) if not provided.

- **data_collator** (`Optional[transformers.DataCollator]`):  
  Data collator to use during training.

- **train_dataset** (`Optional[datasets.Dataset]`):  
  The dataset used for training. It is recommended to use `trl.trainer.ConstantLengthDataset` for dataset creation.

- **eval_dataset** (`Optional[Union[datasets.Dataset, Dict[str, datasets.Dataset]]]`):  
  Dataset used for evaluation. As with the training dataset, it is recommended to use `trl.trainer.ConstantLengthDataset`.

- **tokenizer** (`Optional[transformers.PreTrainedTokenizer]`):  
  Tokenizer for the training process. If not specified, the model's associated tokenizer will be used.

- **model_init** (`Callable[[], transformers.PreTrainedModel]`):  
  A function for model initialization. If none is provided, the default initializer will be used.

- **compute_metrics** (`Callable[[transformers.EvalPrediction], Dict]`, optional):  
  Function to compute metrics during evaluation. Should return a dictionary mapping metric names to values. If not provided, only the loss is computed during evaluation.

- **callbacks** (`List[transformers.TrainerCallback]`):  
  A list of callbacks to be used during training.

- **optimizers** (`Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR]`):  
  The optimizer and learning rate scheduler for the training process.

- **preprocess_logits_for_metrics** (`Callable[[torch.Tensor, torch.Tensor], torch.Tensor]`):  
  A function to preprocess the logits before computing metrics.

- **peft_config** (`Optional[PeftConfig]`):  
  The `PeftConfig` object used to initialize the `PeftModel`.

- **formatting_func** (`Optional[Callable]`):  
  Function used to format data for creating the `ConstantLengthDataset`.

#### Inherits:
- **`transformers.Trainer`**: Inherits all attributes and methods from the `Trainer` class.


In [19]:
from transformers.integrations import MLflowCallback, WandbCallback

In [20]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/anaconda/envs/finetune_env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/anaconda/envs/finetune_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/anaconda/envs/finetune_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

/anaconda/envs/finetune_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [21]:
for cb in trainer.callback_handler.callbacks:
    if isinstance(cb, MLflowCallback) or isinstance(cb, WandbCallback):
        trainer.callback_handler.remove_callback(cb)

In [22]:
# We will also pre-process the model by upcasting the layer norms in float 32 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [23]:
import os

os.environ["DISABLE_MLFLOW_INTEGRATION"] = "TRUE"

In [24]:
trainer.train()

/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.948900
20,1.864000


/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=20, training_loss=1.9064565658569337, metrics={'train_runtime': 259.7225, 'train_samples_per_second': 1.232, 'train_steps_per_second': 0.077, 'total_flos': 2159489078968320.0, 'train_loss': 1.9064565658569337, 'epoch': 0.03249390739236393})

In [25]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/u

<s>[INST] What is a large language model? [/INST] A



In [26]:
result

[{'generated_text': '<s>[INST] What is a large language model? [/INST] A\n'}]